In [45]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, Activation, Dropout
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import pandas as pd
import numpy as np

data_set = pd.read_csv(r"F:\rate_project\source\dataset.csv", sep='\t')
train_y = np_utils.to_categorical(data_set["R"][:15000], 3)
test_y = np_utils.to_categorical(data_set["R"][15000:], 3)
train_x = data_set.drop(columns=["R"])[:15000]
test_x = data_set.drop(columns=["R"])[15000:]

print(train_x.iloc[1000])
print(train_y[1000])

HP-AP         -19.000000
H5LP - A5LP    -8.000000
FF             -0.057692
MHScored        0.880000
MHSkipped       1.480000
MAScored        1.307692
MASkipped       1.076923
Name: 1000, dtype: float64
[0. 0. 1.]


In [34]:
model = Sequential()

model.add(Dense(128, input_shape=(train_x.shape[1], )))
model.add(Activation("relu"))
model.add(Dropout(0.3))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

early_stopping=EarlyStopping(monitor='loss')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_x, train_y,
          epochs=105,
          batch_size=10,
          verbose=1,
          validation_split=0.1,
          callbacks=[early_stopping])

print(model.evaluate(test_x, test_y))


Train on 13500 samples, validate on 1500 samples
Epoch 1/105
13500/13500 [==============================] - 6s 449us/step - loss: 0.8209 - acc: 0.6292 - val_loss: 0.6909 - val_acc: 0.6947
Epoch 2/105
13500/13500 [==============================] - 4s 271us/step - loss: 0.7551 - acc: 0.6596 - val_loss: 0.6907 - val_acc: 0.7060
Epoch 3/105
13500/13500 [==============================] - 4s 276us/step - loss: 0.7450 - acc: 0.6625 - val_loss: 0.6727 - val_acc: 0.7133
Epoch 4/105
13500/13500 [==============================] - 4s 284us/step - loss: 0.7391 - acc: 0.6647 - val_loss: 0.6731 - val_acc: 0.7173
Epoch 5/105
13500/13500 [==============================] - 4s 280us/step - loss: 0.7347 - acc: 0.6647 - val_loss: 0.6779 - val_acc: 0.7167
Epoch 6/105
1646/1646 [==============================] - 0s 51us/step
[0.669098706320743, 0.7071688941443387]


In [54]:
new_train_x = []
new_train_y = []
new_test_x = []
new_test_y = []
tmp_train = train_x.values
tmp_test = test_x.values
for i in range(200, len(tmp_train)):
    new_train_x.append(tmp_train[i-200: i])
    new_train_y.append(train_y[i-1])

for i in range(200, len(tmp_test)):
    new_test_x.append(tmp_test[i-200: i])
    new_test_y.append(test_y[i-1])

new_train_x = np.array(new_train_x)
new_train_y = np.array(new_train_y)
new_test_x = np.array(new_test_x)
new_test_y = np.array(new_test_y)

print(new_train_x.shape, len(new_train_y))
print(len(new_test_x), len(new_test_y))
print(new_train_x[801][199])
print(new_train_y[801])

(14800, 200, 7) 14800
1446 1446
[-19.          -8.          -0.05769231   0.88         1.48
   1.30769231   1.07692308]
[0. 0. 1.]


In [56]:
time_step = 200
input_dim = 7

lstm_model = Sequential()
lstm_model.add(LSTM(32, return_sequences=True, input_shape=(time_step, input_dim)))
lstm_model.add(LSTM(32, return_sequences=True)) 
lstm_model.add(LSTM(32))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

lstm_model.fit(new_train_x, new_train_y,
          epochs=100,
          batch_size=100,
          verbose=1,
          validation_split=0.1,
          callbacks=[early_stopping])


print(lstm_model.evaluate(new_test_x, new_test_y))

Train on 13320 samples, validate on 1480 samples
Epoch 1/100
13320/13320 [==============================] - 96s 7ms/step - loss: 1.0118 - acc: 0.4940 - val_loss: 0.7831 - val_acc: 0.6574
Epoch 2/100
13320/13320 [==============================] - 85s 6ms/step - loss: 0.7756 - acc: 0.6465 - val_loss: 0.7004 - val_acc: 0.6919
Epoch 3/100
13320/13320 [==============================] - 85s 6ms/step - loss: 0.7501 - acc: 0.6572 - val_loss: 0.6886 - val_acc: 0.6966
Epoch 4/100
13320/13320 [==============================] - 85s 6ms/step - loss: 0.7354 - acc: 0.6648 - val_loss: 0.6827 - val_acc: 0.7061
Epoch 5/100
13320/13320 [==============================] - 85s 6ms/step - loss: 0.7275 - acc: 0.6656 - val_loss: 0.6703 - val_acc: 0.7115
Epoch 6/100
13320/13320 [==============================] - 87s 7ms/step - loss: 0.7213 - acc: 0.6716 - val_loss: 0.6722 - val_acc: 0.7108
Epoch 7/100
13320/13320 [==============================] - 85s 6ms/step - loss: 0.7188 - acc: 0.6718 - val_loss: 0.6580 - v